In [1]:
import torch
import sys
import os
from pathlib import Path
import numpy as np
from typing import List
from PIL import Image

sys.path.append(".")
sys.path.append("..")
sys.path.append(os.path.abspath('D:/Docs/Kursach/stylegan3-editing/utils/'))

from models.stylegan3.model import SG3Generator
from common import make_transform, tensor2im
from editor.Editor import FaceEditor
from models.stylegan3.model import GeneratorType

In [2]:
generator = SG3Generator(checkpoint_path="../pretrained_models/stylegan3-r-ffhq-1024x1024.pkl").decoder

num_images_per_cluster = 5
trunc = 0.5

INTERFACEGAN_RANGES = {
    "pose": (-4, 5),
    "age": (-5, 5),
    "smile": (-2, 2),
    "Male": (-2, 4)
}

Loading StyleGAN3 generator from path: ../pretrained_models/stylegan3-r-ffhq-1024x1024.pkl
Done!


In [3]:
if hasattr(generator.synthesis, 'input'):
    m = make_transform(translate=(0, 0), angle=0)
    m = np.linalg.inv(m)
    generator.synthesis.input.transform.copy_(torch.from_numpy(m))

In [4]:
editor = FaceEditor(generator, GeneratorType.ALIGNED)
start_path = Path('./edit_results')

In [5]:
def save_coupled_images(images: List, output_path: Path):
    if type(images[0]) == list:
        images = [image[0] for image in images]
    res = np.array(images[0])
    for image in images[1:]:
        res = np.concatenate([res, image], axis=1)
    res = Image.fromarray(res).convert("RGB")
    res.save(output_path)

In [417]:
z = torch.from_numpy(np.random.randn(1, 512)).to('cuda')
w = generator.mapping(z, None, truncation_psi=trunc)
for direction in ["age", "smile", "pose", "Male"]:
    edit_images, _ = editor.edit(latents=w,
                                 direction=direction,
                                 factor_range=INTERFACEGAN_RANGES[direction],
                                 apply_user_transformations=False)
    save_coupled_images(edit_images[0], output_path=start_path / f"cluster1_{direction}_initial.png")
    save_coupled_images(edit_images[-1], output_path=start_path / f"cluster1_{direction}_final.png")

In [418]:
z = torch.from_numpy(np.random.randn(1, 512)).to('cuda')
w = generator.mapping(z, None, truncation_psi=trunc)
for direction in ["age", "smile", "pose", "Male"]:
    edit_images, _ = editor.edit(latents=w,
                                 direction=direction,
                                 factor_range=INTERFACEGAN_RANGES[direction],
                                 apply_user_transformations=False)
    save_coupled_images(edit_images[0], output_path=start_path / f"cluster2_{direction}_initial.png")
    save_coupled_images(edit_images[-1], output_path=start_path / f"cluster2_{direction}_final.png")